Hello!

In this kernel, i will show you my solution using tensorflow. The task is to classify up to 6 class labels. First of all i get low level feature extracter layers from VGG19 pretrained model, then using fine tuning method add several own fully connected layers with prediction layer.

# import libraries

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

# data loading and data augmentation using Image data generation

I use for augmentation : rotation, shifting to height and width, and flipping images.

In [2]:
training_path = "G:/Desktop/intel image classification/seg_train/seg_train/"
testing_path = "G:/Desktop/intel image classification/seg_test/seg_test/"

I have 14034 images for training, 3000 images for testing.

In [3]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                           zca_whitening=True,
                                                           rotation_range=40,
                                                           width_shift_range=0.15,
                                                           height_shift_range=0.15,
                                                           horizontal_flip=True
                                                           )
training_instances = generator.flow_from_directory(training_path, 
                                                   target_size=(150, 150),
                                                   batch_size=64)

generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
test_instances = generator.flow_from_directory(testing_path,
                                               target_size=(150, 150),
                                               batch_size=32)

C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


# loading pretrained model from internet

With include_top=False parameter i get model without fully connected layers and prediction layer.

In [4]:
IMG_SHAPE = (150, 150, 3)

base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

# in the beginning i will fit only top layers. 

In [5]:
base_model.trainable = False

In [6]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [7]:
len(base_model.layers)

22

Added fully connected layers with prediction softmax layer.

In [8]:
x = base_model.get_layer('block3_pool').output
x = tf.keras.layers.Flatten()(x)

x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(6, activation='softmax')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=prediction)

For learning rate set 0.001

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [12]:
%%time
Batch_size=64
initial_epochs = 20
model_fit=model.fit(training_instances, 
                    steps_per_epoch=14034 // Batch_size,
                    epochs=initial_epochs,
                    validation_data=test_instances
                   )

C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 219 steps, validate for 94 steps
Epoch 1/20


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


219/219 [==============================] - 88s 403ms/step - loss: 111.8320 - accuracy: 0.5895 - val_loss: 14.9559 - val_accuracy: 0.7670
Epoch 2/20
219/219 [==============================] - 87s 399ms/step - loss: 13.9940 - accuracy: 0.7041 - val_loss: 6.6847 - val_accuracy: 0.7713
Epoch 3/20
219/219 [==============================] - 87s 397ms/step - loss: 8.0017 - accuracy: 0.7061 - val_loss: 4.2202 - val_accuracy: 0.7670
Epoch 4/20
219/219 [==============================] - 89s 405ms/step - loss: 3.4411 - accuracy: 0.7175 - val_loss: 1.7073 - val_accuracy: 0.8010
Epoch 5/20
219/219 [==============================] - 86s 395ms/step - loss: 1.9006 - accuracy: 0.7361 - val_loss: 0.9830 - val_accuracy: 0.8170
Epoch 6/20
219/219 [==============================] - 87s 398ms/step - loss: 1.3213 - accuracy: 0.7419 - val_loss: 0.9808 - val_accuracy: 0.7857
Epoch 7/20
219/219 [==============================] - 88s 400ms/step - loss: 1.1389 - accuracy: 0.7452 - val_loss: 0.9827 - val_accuracy:

In [13]:
model.evaluate(test_instances)

  ...
    to  
  ['...']
94/94 [==============================] - 12s 127ms/step - loss: 0.7826 - accuracy: 0.7723


[0.782577098525585, 0.7723333]

After 20 epoch accuracy equal to 77%.

Then apply fine tuning for last 9 layer.

But learning rate will 10 time less than previous value. Because, pretrained model weights already will be closer to desired weights.

In [14]:
base_model.trainable = True

In [15]:
len(base_model.layers)

22

In [17]:
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 7

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  22


In [18]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [21]:
%%time
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  model_fit.epoch[-1],
                         validation_data=test_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 220 steps, validate for 94 steps
Epoch 20/30


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


220/220 [==============================] - 106s 482ms/step - loss: 0.7124 - accuracy: 0.7785 - val_loss: 0.6183 - val_accuracy: 0.7903
Epoch 21/30
220/220 [==============================] - 106s 481ms/step - loss: 0.5025 - accuracy: 0.8185 - val_loss: 0.4129 - val_accuracy: 0.8577
Epoch 22/30
220/220 [==============================] - 105s 477ms/step - loss: 0.4606 - accuracy: 0.8339 - val_loss: 0.4071 - val_accuracy: 0.8613
Epoch 23/30
220/220 [==============================] - 105s 478ms/step - loss: 0.4177 - accuracy: 0.8544 - val_loss: 0.3927 - val_accuracy: 0.8607
Epoch 24/30
220/220 [==============================] - 159s 720ms/step - loss: 0.4063 - accuracy: 0.8535 - val_loss: 0.3507 - val_accuracy: 0.8817
Epoch 25/30
220/220 [==============================] - 248s 1s/step - loss: 0.3832 - accuracy: 0.8626 - val_loss: 0.3284 - val_accuracy: 0.8873
Epoch 26/30
220/220 [==============================] - 366s 2s/step - loss: 0.3565 - accuracy: 0.8754 - val_loss: 0.3165 - val_accura

In [22]:
model.evaluate(test_instances)

  ...
    to  
  ['...']
94/94 [==============================] - 12s 125ms/step - loss: 0.3484 - accuracy: 0.8817


[0.34836831268794993, 0.88166666]

In [23]:
model.save('G:/Desktop/intel image classification/88%.h5')

In [24]:
%%time
fine_tune_epochs1 = 10
total_epochs =  initial_epochs + fine_tune_epochs + fine_tune_epochs1

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  history_fine.epoch[-1],
                         validation_data=test_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 220 steps, validate for 94 steps
Epoch 30/40
220/220 [==============================] - 104s 474ms/step - loss: 0.3101 - accuracy: 0.8879 - val_loss: 0.3269 - val_accuracy: 0.8890
Epoch 31/40
220/220 [==============================] - 105s 475ms/step - loss: 0.2896 - accuracy: 0.8972 - val_loss: 0.2925 - val_accuracy: 0.8990
Epoch 32/40
220/220 [==============================] - 105s 477ms/step - loss: 0.2924 - accuracy: 0.8982 - val_loss: 0.2834 - val_accuracy: 0.8990
Epoch 33/40
220/220 [==============================] - 105s 476ms/step - loss: 0.2767 - accuracy: 0.9010 - val_loss: 0.3354 - val_accuracy: 0.8873
Epoch 34/40
220/220 [==============================] - 105s 476ms/step - loss: 0.2820 - accuracy: 0.8999 - val_loss: 0.3528 - val_accuracy: 0.8837
Epoch 35/40
220/220 [==============================] - 105s 476ms/step - loss: 0.2726 - accuracy: 0.9003 - val_loss: 0.2827 - val_accuracy: 0.9037
Epoch 36/40
220/220 [====

In [25]:
model.evaluate(test_instances)

  ...
    to  
  ['...']
94/94 [==============================] - 17s 181ms/step - loss: 0.3128 - accuracy: 0.8903


[0.31281380910188594, 0.89033335]

After first part fine tuning i will fit again with 18 last layers.

In [30]:
base_model.trainable = True

In [31]:
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 4

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  22


In [32]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [34]:
%%time
fine_tune_epochs2 = 5
total_epochs =  initial_epochs + fine_tune_epochs + fine_tune_epochs1 + fine_tune_epochs2

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  history_fine.epoch[-1],
                         validation_data=test_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 220 steps, validate for 94 steps
Epoch 40/45


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


220/220 [==============================] - 199s 906ms/step - loss: 0.2712 - accuracy: 0.9058 - val_loss: 0.3285 - val_accuracy: 0.9007
Epoch 41/45
220/220 [==============================] - 194s 884ms/step - loss: 0.2619 - accuracy: 0.9056 - val_loss: 0.3622 - val_accuracy: 0.8680
Epoch 42/45
220/220 [==============================] - 165s 752ms/step - loss: 0.2407 - accuracy: 0.9137 - val_loss: 0.2949 - val_accuracy: 0.8933
Epoch 43/45
220/220 [==============================] - 131s 595ms/step - loss: 0.2366 - accuracy: 0.9153 - val_loss: 0.2903 - val_accuracy: 0.9010
Epoch 44/45
220/220 [==============================] - 132s 598ms/step - loss: 0.2285 - accuracy: 0.9186 - val_loss: 0.2803 - val_accuracy: 0.9050
Epoch 45/45
220/220 [==============================] - 131s 597ms/step - loss: 0.2231 - accuracy: 0.9210 - val_loss: 0.3108 - val_accuracy: 0.9043
Wall time: 15min 54s


In [35]:
model.evaluate(test_instances)

  ...
    to  
  ['...']
94/94 [==============================] - 12s 129ms/step - loss: 0.3108 - accuracy: 0.9043


[0.3108464305467428, 0.90433335]

Then i also fit model again with last 20 layers. And each step of fine tuning learning rate value less and less.

In [41]:
base_model.trainable = True

In [42]:
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  22


In [43]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), metrics=["accuracy"])

In [44]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [45]:
%%time
fine_tune_epochs3 = 10
total_epochs =  initial_epochs + fine_tune_epochs + fine_tune_epochs1 + fine_tune_epochs2 + fine_tune_epochs3

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  history_fine.epoch[-1],
                         validation_data=test_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 220 steps, validate for 94 steps
Epoch 45/55


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


220/220 [==============================] - 175s 796ms/step - loss: 0.1762 - accuracy: 0.9370 - val_loss: 0.2644 - val_accuracy: 0.9163
Epoch 46/55
220/220 [==============================] - 173s 788ms/step - loss: 0.1662 - accuracy: 0.9401 - val_loss: 0.2581 - val_accuracy: 0.9143
Epoch 47/55
220/220 [==============================] - 174s 789ms/step - loss: 0.1612 - accuracy: 0.9417 - val_loss: 0.2482 - val_accuracy: 0.9167
Epoch 48/55
220/220 [==============================] - 173s 788ms/step - loss: 0.1516 - accuracy: 0.9451 - val_loss: 0.2510 - val_accuracy: 0.9163
Epoch 49/55
220/220 [==============================] - 174s 792ms/step - loss: 0.1511 - accuracy: 0.9428 - val_loss: 0.2636 - val_accuracy: 0.9157
Epoch 50/55
220/220 [==============================] - 173s 787ms/step - loss: 0.1538 - accuracy: 0.9453 - val_loss: 0.2568 - val_accuracy: 0.9183
Epoch 51/55
220/220 [==============================] - 173s 786ms/step - loss: 0.1481 - accuracy: 0.9458 - val_loss: 0.2547 - val_

In [46]:
model.evaluate(test_instances)

  ...
    to  
  ['...']
94/94 [==============================] - 12s 125ms/step - loss: 0.2514 - accuracy: 0.9157


[0.2513879728008141, 0.91566664]

In [47]:
model.save('G:/Desktop/intel image classification/91%.h5')

Accuracy of last model was 91%.